<a href="https://colab.research.google.com/github/Alan240124/Alan/blob/main/Matricas(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class HeartDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = sorted(os.listdir(image_dir))
        self.masks = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])

        image = Image.open(image_path).convert("L")  # Converter para grayscale
        mask = Image.open(mask_path).convert("L")    # Converter para grayscale

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


# Diretórios das imagens e máscaras
image_dir = '/content/drive/MyDrive/DataSet3/images'  # Substitua pelo caminho real
mask_dir = '/content/drive/MyDrive/DataSet3/masks'    # Substitua pelo caminho real

# Transformação (exemplo: converter para tensor)
#transform = transforms.ToTensor()
# Transforma as imagens para tensor e normaliza
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Redimensione conforme necessário
    transforms.ToTensor(),
])

# Criação do dataset e do DataLoader
dataset = HeartDataset(image_dir, mask_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Exibir algumas informações do dataset
print(f"Total de imagens no dataset: {len(dataset)}")

# Exemplo de iteração pelo DataLoader
for images, masks in dataloader:
    print(f"Tamanho do batch de imagens: {images.shape}")
    print(f"Tamanho do batch de máscaras: {masks.shape}")
    break  # Apenas um batch para demonstração

Total de imagens no dataset: 9
Tamanho do batch de imagens: torch.Size([4, 1, 256, 256])
Tamanho do batch de máscaras: torch.Size([4, 1, 256, 256])


In [2]:
import torch.nn as nn
import torch

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )

        self.encoder1 = conv_block(1, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.encoder2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.encoder3 = conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.encoder4 = conv_block(256, 512)
        self.pool4 = nn.MaxPool2d(2)

        self.bottleneck = conv_block(512, 1024)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = conv_block(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = conv_block(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = conv_block(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = conv_block(128, 64)

        self.conv_last = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(self.pool1(e1))
        e3 = self.encoder3(self.pool2(e2))
        e4 = self.encoder4(self.pool3(e3))

        b = self.bottleneck(self.pool4(e4))

        d4 = self.upconv4(b)
        d4 = torch.cat((d4, e4), dim=1)
        d4 = self.decoder4(d4)
        d3 = self.upconv3(d4)
        d3 = torch.cat((d3, e3), dim=1)
        d3 = self.decoder3(d3)
        d2 = self.upconv2(d3)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.decoder2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.decoder1(d1)

        return torch.sigmoid(self.conv_last(d1))

# Inicializar o modelo e definir o otimizador e a função de perda
model = UNet()
criterion = nn.BCELoss()  # Usamos BCELoss para segmentação binária
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [5]:
# Configuração do dispositivo (GPU se disponível)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loop de treinamento
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, masks in dataloader:
        images = images.to(device)
        masks = masks.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader)}")

    # Salva o modelo após cada epoch
    torch.save(model.state_dict(), f"unet_epoch_{epoch+1}.pth")

Epoch [1/10], Loss: 0.09899218877156575
Epoch [2/10], Loss: 0.0978844886024793
Epoch [3/10], Loss: 0.09699309120575587
Epoch [4/10], Loss: 0.1113407239317894
Epoch [5/10], Loss: 0.09998705983161926
Epoch [6/10], Loss: 0.09798662116130193
Epoch [7/10], Loss: 0.10053092737992604
Epoch [8/10], Loss: 0.09873532503843307
Epoch [9/10], Loss: 0.0925753836830457
Epoch [10/10], Loss: 0.08920158445835114


In [7]:
def dice_coefficient(pred, target, epsilon=1e-6):
    intersection = (pred * target).sum()
    return (2. * intersection + epsilon) / (pred.sum() + target.sum() + epsilon)

# Exemplo de uso com uma batch do DataLoader
model.eval()
with torch.no_grad():
    for images, masks in dataloader:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        dice = dice_coefficient(outputs, masks)
        print("DICE coefficient:", dice.item())
        break  # Apenas para uma batch

DICE coefficient: 0.8818418979644775


In [9]:
import torch
import numpy as np

# Função de cálculo do DICE coefficient
def dice_coefficient(pred, target, epsilon=1e-6):
    pred = (pred > 0.5).float()  # Binarizar a predição
    target = (target > 0.5).float()  # Binarizar a máscara
    intersection = (pred * target).sum()  # Calcular a interseção
    return (2. * intersection + epsilon) / (pred.sum() + target.sum() + epsilon)  # Fórmula do DICE

# Função de cálculo da Intersection over Union (IoU)
def iou_score(pred, target, epsilon=1e-6):
    pred = (pred > 0.5).float()
    target = (target > 0.5).float()
    intersection = (pred * target).sum()  # Interseção
    union = pred.sum() + target.sum() - intersection  # União
    return (intersection + epsilon) / (union + epsilon)  # Fórmula do IoU

# Função de cálculo da Accuracy
def accuracy_score(pred, target):
    pred = (pred > 0.5).float()  # Binarizar a predição
    target = (target > 0.5).float()  # Binarizar a máscara
    correct = (pred == target).sum()  # Contar acertos
    total = target.numel()  # Total de pixels
    return correct / total  # Proporção de acertos

# Exemplo de uso com uma batch do DataLoader
model.eval()  # Coloca o modelo em modo de avaliação
with torch.no_grad():  # Desliga o cálculo de gradientes
    for images, masks in dataloader:  # Loop sobre as batches
        images = images.to(device)  # Mover para o dispositivo (GPU ou CPU)
        masks = masks.to(device)

        # Passando as imagens pela rede
        outputs = model(images)

        # Calcular as métricas
        dice = dice_coefficient(outputs, masks)  # DICE coefficient
        iou = iou_score(outputs, masks)  # IoU
        accuracy = accuracy_score(outputs, masks)  # Accuracy

        # Exibindo os resultados
        print(f"DICE coefficient: {dice.item():.4f}")
        print(f"IoU score: {iou.item():.4f}")
        print(f"Accuracy: {accuracy.item():.4f}")

        break  # Apenas para uma batch, remova esta linha para calcular para todas as batches


DICE coefficient: 0.9163
IoU score: 0.8456
Accuracy: 0.9706
